# Fast BBH analysis

This notebooks demonstrates analysing a subset of the standard BBH parameters using the importance nested sampler in `nessai`.

This example is based on [`fast_tutorial.py`](https://git.ligo.org/lscsoft/bilby/-/blob/master/examples/gw_examples/injection_examples/fast_tutorial.py) in `bilby`.

## Installation

We need to install bleeding edge versions of `bilby` and `nessai` to use the importance nested sampler in `bilby`.

**Warning:** these versions of the packages are bleeding edge and subject to changes.

In [ ]:
!pip install git+https://github.com/mj-will/nessai.git@stable-ins --no-deps
!pip install git+https://git.ligo.org/michael.williams/bilby.git@nessai-update-for-ins --no-deps --force

## Setup

In [ ]:
import bilby
import numpy as np

This example we're going to analyse a 4-second signal sampled at 2kHz.

In [ ]:
duration = 4.0
sampling_frequency = 2048.0
minimum_frequency = 20

outdir = "outdir"
label = "gw_ins_nessai"
bilby.core.utils.setup_logger(outdir=outdir, label=label)


## Injection

This example uses the GW150914-like injection from the original bilby example.

In [ ]:
injection_parameters = dict(
    mass_1=36.0,
    mass_2=29.0,
    a_1=0.4,
    a_2=0.3,
    tilt_1=0.5,
    tilt_2=1.0,
    phi_12=1.7,
    phi_jl=0.3,
    luminosity_distance=2000.0,
    theta_jn=0.4,
    psi=2.659,
    phase=1.3,
    geocent_time=1126259642.413,
    ra=1.375,
    dec=-1.2108,
)

## Likelihood


We set up the waveform generator and interferometers. and the define the likelihood. In this case we're going to use a three detector network.

In [ ]:
waveform_arguments = dict(
    waveform_approximant="IMRPhenomPv2",
    reference_frequency=50.0,
    minimum_frequency=minimum_frequency,
)

waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments,
)

# Set up a three-detector network.
ifos = bilby.gw.detector.InterferometerList(["H1", "L1", "V1"])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=injection_parameters["geocent_time"] - 2,
)
ifos.inject_signal(
    waveform_generator=waveform_generator, parameters=injection_parameters
)

likelihood = bilby.gw.GravitationalWaveTransient(
    interferometers=ifos, waveform_generator=waveform_generator
)


## Priors

We then define the priors and fix a number of the parameters. 

We're going to sample chirp_mass, mass ratio, luminosity distance and inclination. The priors will be set to the defaults in bilby for precessing BBHs.

In [ ]:
priors = bilby.gw.prior.BBHPriorDict()
for key in [
    "a_1",
    "a_2",
    "tilt_1",
    "tilt_2",
    "phi_12",
    "phi_jl",
    "psi",
    "ra",
    "dec",
    "geocent_time",
    "phase",
]:
    priors[key] = injection_parameters[key]

# Perform a check that the prior does not extend to a parameter space longer than the data
priors.validate_prior(duration, minimum_frequency)


## Running the sampler

The importance nested sampler in `nessai` is implmented as a different sampler in bilby. So we set `sampler="nessai_importance"`. The main differences when calling the sampler are the optional keyword arguments that can be specified.

**Important:** the defaults are subject to change so the exact wall time and results may change.

In [ ]:

result = bilby.run_sampler(
    likelihood=likelihood,
    priors=priors,
    sampler="nessai_importance",
    nlive=1000,
    injection_parameters=injection_parameters,
    outdir=outdir,
    label=label,
    clean=True,
    conversion_function=bilby.gw.conversion.generate_all_bbh_parameters,
)


## Posterior distribution

In [ ]:
result.plot_corner()